In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from xml.dom.minidom import parse, parseString

# SFRA Test

In [ ]:
DIR_PATH = "data/pltu_priok_bbt/"
DATA = "bc00713c-c3ba-4d93-b514-aa2400ebfb57.xml"

In [ ]:
dom = parse(DIR_PATH + DATA)
dom

In [ ]:
import re

def parse_string(string):
    string = re.sub('\d', '', string)
    string = string.replace(": ", "")
    string = string.replace(" ", "_")
    return string

In [ ]:
# ambil seluruh nama data
fra_measurements = dom.getElementsByTagName("FRAMeasurement")

df = pd.DataFrame()
for i in range(len(fra_measurements)):
    measurement = parse_string(fra_measurements[i].getElementsByTagName("Name")[0].firstChild.nodeValue)
    data_measure = fra_measurements[i].getElementsByTagName("FRASweepPoint")
    
    freq_l, mag_l, phase_l = [], [], []
    for data in data_measure:
        freq = data.getElementsByTagName("Frequency")[0].firstChild.nodeValue
        mag = data.getElementsByTagName("Magnitude")[0].firstChild.nodeValue
        phase = data.getElementsByTagName("Phase")[0].firstChild.nodeValue
        
        freq_l.append(freq)
        mag_l.append(mag)
        phase_l.append(phase)
        
    df[str(measurement) + "_freq"] = pd.to_numeric(freq_l)
    df[str(measurement) + "_mag"] = pd.to_numeric(mag_l)
    df[str(measurement) + "_phase"] = pd.to_numeric(phase_l)

In [ ]:
df.head()

In [ ]:
df.describe()

## Plotting

In [ ]:
df.columns

In [ ]:
df_sfra = df[df.columns[3:9]]
df_sfra['deviasi'] = df_sfra['A_B_mag'] - df_sfra['B_C_mag']
df_sfra['A_B_freq_log'] = np.log(df_sfra['A_B_mag'])
df_sfra['B_C_freq_log'] = np.log(df_sfra['B_C_mag'])
df_sfra.head()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(np.log(df["A_B_freq"]), df["A_B_mag"], color='red')
plt.plot(np.log(df["B_C_freq"]), df["B_C_mag"], color='blue')

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(np.log(df["A_B_freq"]), df["A_B_mag"]-df["B_C_mag"], color='red')
# plt.plot(np.log(df["B_C_freq"]), df["B_C_phase"], color='blue')

In [ ]:
np.log(1000)

# Winding Resistance (LV Winding)

In [ ]:
DIR_PATH = "data/pltu_priok_bbt/"
DATA = "30749e27-460f-479f-8356-aa2a01043583.xml"

In [ ]:
dom_wr_lv = parse(DIR_PATH + DATA)
dom_wr_lv

In [ ]:
# ambil seluruh nama data
wr_sec_measurements = dom_wr_lv.getElementsByTagName("TransformerWindingResistanceMeasurement")

df_wr_lv = pd.DataFrame()
r_value_l, r_dev_l = [], []
s_value_l, s_dev_l = [], []
t_value_l, t_dev_l = [], []

for i in range(len(wr_sec_measurements)):
    phasa_r = wr_sec_measurements[i].getElementsByTagName("PhaseAResult")
    phasa_s = wr_sec_measurements[i].getElementsByTagName("PhaseBResult")
    phasa_t = wr_sec_measurements[i].getElementsByTagName("PhaseCResult")
     
    # phasa r
    r_value = phasa_r[0].getElementsByTagName("ResistanceMeasured")[0].firstChild.nodeValue
    r_dev = phasa_r[0].getElementsByTagName("Deviation")[0].firstChild.nodeValue
    
    # phasa s
    s_value = phasa_s[0].getElementsByTagName("ResistanceMeasured")[0].firstChild.nodeValue
    s_dev = phasa_s[0].getElementsByTagName("Deviation")[0].firstChild.nodeValue

    # phasa t
    t_value = phasa_t[0].getElementsByTagName("ResistanceMeasured")[0].firstChild.nodeValue
    t_dev = phasa_t[0].getElementsByTagName("Deviation")[0].firstChild.nodeValue

    r_value_l.append(r_value)
    r_dev_l.append(r_dev)
    s_value_l.append(s_value)
    s_dev_l.append(s_dev)
    t_value_l.append(t_value)
    t_dev_l.append(t_dev)

        
df_wr_lv["phasa_r"] = r_value_l
df_wr_lv["dev_r"] = r_dev_l
df_wr_lv["phasa_s"] = s_value_l
df_wr_lv["dev_s"] = s_dev_l
df_wr_lv["phasa_t"] = t_value_l
df_wr_lv["dev_t"] = t_dev_l

# drop NaN
df_wr_lv = df_wr_lv.apply (pd.to_numeric, errors='coerce')
df_wr_lv.dropna()

In [ ]:
df_wr_lv.dtypes

# Winding Resistance HV Winding

In [ ]:
DIR_PATH = "data/pltu_priok_bbt/"
DATA = "b2510b5f-1243-445d-a39e-aa2a00e47eee.xml"

In [ ]:
# ambil seluruh nama data
dom_wr_hv = parse(DIR_PATH + DATA)
wr_prim_measurements = dom_wr_hv.getElementsByTagName("TransformerWindingResistanceMeasurement")

df_wr_hv = pd.DataFrame()
tap_name_l, direction_l = [], []
r_value_l, r_dev_l = [], []
s_value_l, s_dev_l = [], []
t_value_l, t_dev_l = [], []

for i in range(len(wr_prim_measurements)):
    tap_name = wr_prim_measurements[i].getElementsByTagName("TapName")[0].firstChild.nodeValue
    direction = wr_prim_measurements[i].getElementsByTagName("Direction")[0].firstChild.nodeValue
    phasa_r = wr_prim_measurements[i].getElementsByTagName("PhaseAResult")
    phasa_s = wr_prim_measurements[i].getElementsByTagName("PhaseBResult")
    phasa_t = wr_prim_measurements[i].getElementsByTagName("PhaseCResult")
    
    # phasa r
    r_value = phasa_r[0].getElementsByTagName("ResistanceMeasured")[0].firstChild.nodeValue
    r_dev = phasa_r[0].getElementsByTagName("Deviation")[0].firstChild.nodeValue
    
    # phasa s
    s_value = phasa_s[0].getElementsByTagName("ResistanceMeasured")[0].firstChild.nodeValue
    s_dev = phasa_s[0].getElementsByTagName("Deviation")[0].firstChild.nodeValue

    # phasa t
    t_value = phasa_t[0].getElementsByTagName("ResistanceMeasured")[0].firstChild.nodeValue
    t_dev = phasa_t[0].getElementsByTagName("Deviation")[0].firstChild.nodeValue
    
    tap_name_l.append(tap_name)
    direction_l.append(direction)
    r_value_l.append(r_value)
    r_dev_l.append(r_dev)
    s_value_l.append(s_value)
    s_dev_l.append(s_dev)
    t_value_l.append(t_value)
    t_dev_l.append(t_dev)

df_wr_hv["tap_name"] = tap_name_l
df_wr_hv["direction"] = direction_l
df_wr_hv["phasa_r"] = r_value_l
df_wr_hv["dev_r"] = r_dev_l
df_wr_hv["phasa_s"] = s_value_l
df_wr_hv["dev_s"] = s_dev_l
df_wr_hv["phasa_t"] = t_value_l
df_wr_hv["dev_t"] = t_dev_l

In [ ]:
df_wr_hv